In [38]:
import pandas as pd
ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
dates = pd.read_csv('data/dates.csv', sep=',')
movies = pd.read_csv('data/movies.csv', sep=',')
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')

In [12]:
ratings1_df = ratings1.copy()

In [13]:
ratings2_df = ratings2.copy()

In [14]:
dates_df = dates.copy()

In [15]:
movies_df = movies.copy()

In [39]:
orders_df = orders.copy()
products_df = products.copy()

In [ ]:
# Задание 5.2
movies_df['movieId'].nunique()
# Задание 5.3
ratings1_df['userId'].nunique()
# Задание 5.4
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.value_counts()

In [ ]:
#одинаковое количество столбцов и имена, таблица прибавляется снизу
# также замена индексов на новые, чтобы не было дублей
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

In [ ]:
#удаляем дубли
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

In [ ]:
# соединяем таблицы, выводим семь последних строк
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

In [21]:
# Задание 6.3
def concat_user_files(path):
    data = pd.DataFrame()
    files = os.listdir(path)
    files.sort()
    for file in files:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data])
    data = data.drop_duplicates(ignore_index=True)
    return data

In [ ]:
# объединим таблицы типом left. Так как в наших таблицах есть 
# одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки
# здесь не используется указание на столбец, с помощью которого таблицы объединяются
joined_false = ratings_dates.join(movies, rsuffix='_right', how='left')
display(joined_false)

In [ ]:
joined = ratings_dates.join(movies.set_index('movieId'), on='movieId', how='left')
display(joined.head())

In [ ]:
merged = ratings_dates.merge(movies, on='movieId', how='left')
display(merged.head())

In [ ]:
# Задание 7.5
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged = items_df.merge(purchase_df, on='item_id', how='inner')
income = sum(merged['price']*merged['stock_count'])
display(merged)

In [26]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [ ]:
# Задание 8.1
joined['year_release'] = joined['title'].apply(get_year_release)
print(int(joined['year_release'].shape[0]) - int(joined['year_release'].count()))

In [ ]:
# Задание 8.2
print(joined[joined['year_release'] == 1999.0].groupby('title')['rating'].mean().sort_values())

In [ ]:
# Задание 8.3
print(joined[joined['year_release'] == 2010.0].groupby('genres')['rating'].mean().sort_values())

In [ ]:
# Задание 8.4
print(joined.groupby('userId')['genres'].nunique().sort_values(ascending=False))

In [ ]:
# Задание 8.5
print(joined.groupby('userId')['rating'].agg(['count', 'mean']
        ).sort_values(['count', 'mean'], ascending=[True, False]))

In [ ]:
# Задание 8.6
temp = joined[joined['year_release'] == 2018.0].groupby('genres'
        )['rating'].agg(['count', 'mean'])
result = temp[temp['count'] >10].sort_values(by = 'mean', ascending=False)
print(result)

In [ ]:
# Задание 8.7
joined['year_rating'] = joined['date'].dt.year
result = joined.pivot_table(values='rating', index='year_rating', columns='genres',
                aggfunc='mean')
print(result[result['Action|Adventure'] < 3])
print(result['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values())
print(result['Animation|Children|Mystery'].loc[2018])
print(result['Comedy'])

In [ ]:
# Задание 8.8
orders_products = orders_df.merge(products_df, left_on='ID товара', right_on='Product_ID', how='left')
print(orders_products[orders_products['Name'].isna()])

In [84]:
# Задание 8.9
print(orders_products[orders_products['Отменен'] == 'Да'])

         Дата создания  Order ID  ID Покупателя   Статус Оплачен Отменен  \
6  08.11.2019 08:36:22         5              5  Отменён     Нет      Да   

  Отгружен  ID товара  Количество  Product_ID                  Name  Price  \
6      Нет        124           1       124.0  Носки беговые Camino  999.0   

  CURRENCY  
6      RUR  


In [ ]:
# Задание 8.10
print((orders_products[orders_products['Оплачен'] == 'Да']['Price']*orders_products[orders_products['Оплачен'] == 'Да']['Количество']).sort_values())
print(orders_products.loc[10])

In [ ]:
# Задание 8.10
print(orders_products.info())